<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.883 · Aprendizaje por refuerzo</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Ejemplo: Método _TD learning_ en el entorno WindyGridWorld

En este ejemplo implementaremos el método _TD learning_ de aprendizaje por refuerzo para buscar una solución óptima en el problema de WindyGridWorld.

## 1. El entorno __WindyGridWorld__

El entorno __WindyGridWorld__ consiste en un agente que se mueve en una cuadrícula 7x10 (alto x ancho). En cada paso, el agente tiene 4 opciones de acción o movimiento: ARRIBA, ABAJO, DERECHA, IZQUIERDA. El agente siempre sale de la misma casilla [3, 0] y el juego termina cuando el agente llega a la casilla de llegada [3, 7]. 

El entorno se corresponde con el ejemplo 'Cuadrícula con viento' explicado en la sección 3.1.2. el módulo "Métodos de Diferencia Temporal". El problema radica en que hay un viento que empuja al agente hacia arriba en la parte central de la cuadrícula. Esto provoca que, aunque se ejecute una acción estándar, en la región central los estados resultantes se desplazan hacia arriba por un viento cuya fuerza varía entre columnas.

<img src="../figs/GridWorld.png">

El código para implementar este entorno, que se encuentra disponible en el fichero adjunto `windy_gridworld_env.py`, ha sido adaptado del siguiente enlace:

https://pypi.org/project/gym-gridworlds/

## 2. Métodos de Diferencia Temporal

El objetivo de este ejercicio es realizar una estimación de la política óptima mediante los métodos de Diferencia Temporal en el entorno WindyGridWorld.

A continuación, implementaremos el algoritmo de *Q-learning* explicado en el modulo "Aprendizaje por Diferencia Temporal" utilizando los siguientes parámetros:
    
- número de episodios = 200
- *learning rate* = 0.5
- *discount factor* = 1
- *epsilon* = 0.05    

Además, queremos que la solución propuesta nos permita: 

1. Mostrar por pantalla los valores Q estimados para cada estado. 
2. Mostrar por pantalla los valores de la función de valor $v_\pi(s)$ estimada para cada estado. 
3. Ejecutar un episodio siguiendo la política óptima encontrada, donde se pueda reconocer la trayectoria seguida por el agente. 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.collections as mcol
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D
from collections import defaultdict
import gym
import numpy as np
import windy_gridworld_env as wge

env = wge.WindyGridworldEnv()
print("Action space is {} ".format(env.action_space))
print("Observation space is {} ".format(env.observation_space))
print("Reward range is {} ".format(env.reward_range))

Action space is Discrete(4) 
Observation space is Tuple(Discrete(7), Discrete(10)) 
Reward range is (-inf, inf) 


In [2]:
def epsilon_greedy_policy(Q, state, nA, epsilon):
    '''
    Create a policy in which epsilon dictates how likely it will 
    take a random action.

    :param Q: links state -> action value (dictionary)
    :param state: state character is in (int)
    :param nA: number of actions (int)
    :param epsilon: chance it will take a random move (float)
    :return: probability of each action to be taken (list)
    '''
    probs = np.ones(nA) * epsilon / nA
    best_action = np.argmax(Q[state])
    probs[best_action] += 1.0 - epsilon

    return probs

In [3]:
def Q_learning(episodes, learning_rate, discount, epsilon):
    '''
    Learn to solve the environment using Q-learning

    :param episodes: Number of episodes to run (int)
    :param learning_rate: How fast it will converge to a point (float [0, 1])
    :param discount: How much future events lose their value (float [0, 1])
    :param epsilon: chance a random move is selected (float [0, 1])
    :return: x,y points to graph
    '''

    # Links state to action values
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    # Points to plot
    # number of episodes
    x = np.arange(episodes)
    # Number of steps
    y = np.zeros(episodes)
    
    for episode in range(episodes):
        state = env.reset()
                
        for step in range(10000):

            # Select and take action
            probs = epsilon_greedy_policy(Q, state, env.action_space.n, epsilon)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, done, _ = env.step(action)
           
            # TD Update
            td_target = reward + discount * np.amax(Q[next_state])
            td_error = td_target - Q[state][action]
            Q[state][action] += learning_rate * td_error
                        
            if done:
                y[episode] = step
                break

            state = next_state   
                       
    return x, y, Q

In [4]:
x, y, q = Q_learning(episodes=200, learning_rate=0.5, discount=1, epsilon=0.05)

In [5]:
#mostrar los Q de cada estado-acción y la función valor

f = open('output.txt', 'w')
for i in range(7):
    for j in range(10):
        V = max(q[i,j])    
        print((i,j), q[i,j], V)
        print((i,j), q[i,j], V, file=f)

(0, 0) [-13.5        -13.56817121 -13.99884244 -13.42727579] -13.427275786779001
(0, 1) [-13.35708173 -13.42477065 -13.47805434 -13.3875045 ] -13.357081732283543
(0, 2) [-13.1960478  -12.9086274  -13.24240882 -13.11640552] -12.908627395059405
(0, 3) [-12.         -11.99598565 -12.         -12.8421897 ] -11.995985649026961
(0, 4) [-11.63301402 -10.99999999 -11.69134837 -11.60436134] -10.999999990312016
(0, 5) [-10.5        -10.         -10.5        -10.08108124] -10.0
(0, 6) [ -9.49998088  -9.          -9.875      -10.71021865] -9.0
(0, 7) [-8.87469142 -8.         -8.         -9.21932174] -8.0
(0, 8) [-7.98046875 -7.         -7.36194229 -8.52707291] -7.0
(0, 9) [-6.3198061  -6.9375     -6.         -7.96594195] -6.0
(1, 0) [-13.91727947 -14.06284124 -14.32849942 -13.94065094] -13.917279473729955
(1, 1) [-13.62623572 -13.25506492 -13.85403354 -13.76927917] -13.255064916352932
(1, 2) [-13.20183347 -12.88266899 -12.93578341 -12.9130946 ] -12.882668989094089
(1, 3) [-12.15379363 -11.99835672

In [6]:
# función que muestra los valores de la función de estado V(s) en la cuadricula

def print_values(Q, height, width):
    for i in range(height):
        print("------------------------------------------------------------------------------------------")
        for j in range(width):
            arr = np.array(Q[i,j])
            v = np.amax(arr)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")

In [7]:
print_values(q, 7, 10)

------------------------------------------------------------------------------------------
-13.43|-13.36|-12.91|-12.00|-11.00|-10.00|-9.00|-8.00|-7.00|-6.00|
------------------------------------------------------------------------------------------
-13.92|-13.26|-12.88|-12.00|-11.00|-10.00|-9.00|-7.97|-6.00|-5.00|
------------------------------------------------------------------------------------------
-14.36|-13.80|-12.99|-12.00|-11.00|-10.00|-9.00|-4.37|-5.98|-4.00|
------------------------------------------------------------------------------------------
-15.00|-14.00|-13.00|-12.00|-10.99|-9.97|-8.86| 0.00|-3.31|-3.00|
------------------------------------------------------------------------------------------
-14.22|-13.62|-12.86|-11.97|-10.81|-9.67| 0.00| 0.00|-1.00|-2.00|
------------------------------------------------------------------------------------------
-13.61|-12.99|-12.34|-11.71|-10.45| 0.00| 0.00| 0.00|-0.88|-1.55|
-------------------------------------------------------

In [8]:
# ejecución de un episodio siguiendo la politica optima

def execute_episode_TD(q, env):
    obs = env.reset()
    t, total_reward, done = 0, 0, False

    print("Obs inicial: {} ".format(obs))

    switch_action = {
            0: "U",
            1: "R",
            2: "D",
            3: "L",
        }

    for t in range(1000): # limitamos el número de time-steps de cada episodio a 1000
        
        # Elegir una acción siguiendo la política óptima
        arr = np.array(q[obs])
        action = arr.argmax()
       
        # Ejecutar la acción y esperar la respuesta del entorno
        new_obs, reward, done, info = env.step(action)
        obs = new_obs
        print("Action: {} -> Obs: {} and reward: {}".format(switch_action[action], obs, reward))

        if t==999:
            print("Number of time-septs exceeds 1000. STOP episode.") 
        total_reward += reward
        t += 1
        if done:
            break
   
    print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
    env.close()

In [9]:
execute_episode_TD(q, env)

Obs inicial: (3, 0) 
Action: R -> Obs: (3, 1) and reward: -1
Action: R -> Obs: (3, 2) and reward: -1
Action: R -> Obs: (3, 3) and reward: -1
Action: R -> Obs: (2, 4) and reward: -1
Action: R -> Obs: (1, 5) and reward: -1
Action: R -> Obs: (0, 6) and reward: -1
Action: R -> Obs: (0, 7) and reward: -1
Action: R -> Obs: (0, 8) and reward: -1
Action: R -> Obs: (0, 9) and reward: -1
Action: D -> Obs: (1, 9) and reward: -1
Action: D -> Obs: (2, 9) and reward: -1
Action: D -> Obs: (3, 9) and reward: -1
Action: D -> Obs: (4, 9) and reward: -1
Action: L -> Obs: (4, 8) and reward: -1
Action: L -> Obs: (3, 7) and reward: -1
Episode finished after 15 timesteps and reward was -15 
